In [1]:
excluded_session = 1

In [2]:
%env WANDB_PROJECT = final_model_

env: WANDB_PROJECT=final_model_


In [3]:
model_ckp = "bert-base-uncased"
batch_size = 32

In [25]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.5 MB/s eta 0:00:00


In [4]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: minooshayan97 (minoosh). Use `wandb login --relogin` to force relogin


True

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [29]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP_Text_dataset")

  0%|          | 0/5 [00:00<?, ?it/s]

In [33]:
dataset

DatasetDict({
    Session1: Dataset({
        features: ['TURN_NAME', 'emotion', 'transcription'],
        num_rows: 1085
    })
    Session2: Dataset({
        features: ['TURN_NAME', 'emotion', 'transcription'],
        num_rows: 1023
    })
    Session3: Dataset({
        features: ['TURN_NAME', 'emotion', 'transcription'],
        num_rows: 1151
    })
    Session4: Dataset({
        features: ['TURN_NAME', 'emotion', 'transcription'],
        num_rows: 1031
    })
    Session5: Dataset({
        features: ['TURN_NAME', 'emotion', 'transcription'],
        num_rows: 1241
    })
})

In [34]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckp)

In [35]:
def preprocess_function(examples):
    return tokenizer(examples["transcription"], truncation=True)

In [36]:
dataset=dataset.remove_columns(['TURN_NAME'])

In [37]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

In [38]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [39]:
import evaluate

accuracy = evaluate.load("accuracy")

In [40]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
labels = dataset["Session1"].features["emotion"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["1"]

'hap'

In [41]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckp, num_labels=4, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [42]:
tokenized_dataset = tokenized_dataset.rename_column("emotion", "label")

In [47]:
ds1 = tokenized_dataset['Session1'].train_test_split(test_size=0.2)
ds2 = tokenized_dataset['Session2'].train_test_split(test_size=0.2)
ds3 = tokenized_dataset['Session3'].train_test_split(test_size=0.2)
ds4 = tokenized_dataset['Session4'].train_test_split(test_size=0.2)
ds5 = tokenized_dataset['Session5'].train_test_split(test_size=0.2)

In [46]:
ds2

DatasetDict({
    train: Dataset({
        features: ['label', 'transcription', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 818
    })
    test: Dataset({
        features: ['label', 'transcription', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 205
    })
})

In [50]:
import torch
train_ds = torch.utils.data.ConcatDataset([#ds1['train'], 
                                           ds2['train'],
                                           ds3['train'],
                                           ds4['train'],
                                           ds5['train']])

In [49]:
val_ds = torch.utils.data.ConcatDataset([#ds1['test'],
                                         ds2['test'],
                                         ds3['test'],
                                         ds4['test'],
                                         ds5['test']])

In [51]:
training_args = TrainingArguments(
    output_dir="finetuned_bert-base-on-IEMOCAP_1",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=10,
    num_train_epochs=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to='wandb',
    run_name='finetune_bert_base_on_IEMOCAP_1'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/minoosh/finetuned_bert-base-on-IEMOCAP_1 into local empty directory.


In [52]:
import torch
torch.cuda.empty_cache()

In [53]:
results = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.290900,1.220701,0.531390
2,0.932800,0.923434,0.644619
3,0.757200,0.775114,0.704036
4,0.568600,0.770758,0.717489
5,0.379600,0.825850,0.713004
6,0.273600,0.862046,0.723094
7,0.218200,0.893917,0.723094
8,0.175500,1.078757,0.721973
9,0.151400,1.002873,0.736547
10,0.166500,1.081905,0.715247


In [54]:
trainer.evaluate(tokenized_dataset['Session1'])

{'eval_loss': 1.3270843029022217,
 'eval_accuracy': 0.6516129032258065,
 'eval_runtime': 3.2666,
 'eval_samples_per_second': 332.149,
 'eval_steps_per_second': 10.408,
 'epoch': 20.0}

In [55]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/minoosh/finetuned_bert-base-on-IEMOCAP_1
   6f90eb8..9cb3812  main -> main

   6f90eb8..9cb3812  main -> main

To https://huggingface.co/minoosh/finetuned_bert-base-on-IEMOCAP_1
   9cb3812..dd5d4ad  main -> main

   9cb3812..dd5d4ad  main -> main



'https://huggingface.co/minoosh/finetuned_bert-base-on-IEMOCAP_1/commit/9cb3812ef1d2417d6d021aeadcb47eac8e66bb2e'